## Setup

In [1]:
##### Setup #####
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Load the data
download the zip file with terminal
* curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
* unzip -q kagglecatsanddogs_3367a.zip
* ls